In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
from catboost import CatBoostRegressor
from sklearn.decomposition import PCA  # Principal component

from functions.datacleaning import MoscowHousing as mh
from functions.distance import get_distance_coordinates, norm_features

import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')

# Prepare NaN Values

In [2]:
# Get data
data = mh(
          coordinates=None,
          data_train="../prepared_data/train_prepared.csv",
          data_test="../prepared_data/test_prepared.csv",
          need_correction=False,
          normalize = False,
          features_float = ["area_total", 
                            "distance", 
                            "area_kitchen", 
                            "area_living", 
                            "ceiling",
                            "distance_metro"]
          )

In [3]:
XTrain, YTrain = data.get_data_train()
XTest = data.get_data_test()

In [15]:
XTrain.head()

,building_id,id,seller,price,area_total,price_per_square_meter,area_kitchen,area_living,floor,rooms,...,material,stories,elevator_without,elevator_passenger,elevator_service,parking,garbage_chute,heating,distance,distance_metro
0,0,18450,NaN,11300000.0,-0.011096,153116.531165,-0.541037,0.263027,1,3,...,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,10.252880,286.45
1,1,20065,1.0,7100000.0,-0.553098,169047.619048,-0.388730,-0.596888,5,1,...,2.0,17,0.0,1.0,1.0,1.0,1.0,0.0,36.091930,16183.78
2,2,2137,0.0,13600000.0,0.009357,181333.333333,-0.388730,0.517816,3,3,...,3.0,12,0.0,1.0,0.0,1.0,1.0,0.0,12.374064,956.44
3,3,4155,1.0,10100000.0,-0.263348,171186.440678,NaN,NaN,2,2,...,2.0,7,0.0,1.0,1.0,NaN,NaN,0.0,15.232969,1405.21
4,3,8798,1.0,10100000.0,-0.263348,171186.440678,-0.084116,-0.262477,2,2,...,2.0,7,0.0,1.0,1.0,NaN,NaN,0.0,15.232969,1405.21


In [5]:
print(YTrain)

0        11300000.0
1         7100000.0
2        13600000.0
3        10100000.0
4        10100000.0
            ...    
23280    11229240.0
23281    10047510.0
23282     7898040.0
23283    12442250.0
23284     4999500.0
Name: price, Length: 23285, dtype: float64


In [6]:
print(XTest)

      building_id     id  seller  area_total  area_kitchen  area_living  \
0               4  26052     NaN    5.675295           NaN          NaN   
1               5  29404     1.0   -0.896821     -1.581108    -0.588336   
2               6  24089     2.0    0.494253     -0.417982     0.678815   
3              11  24033     0.0    0.087932      0.163582     0.165921   
4              11  32976     1.0    0.096804     -0.316208     0.368062   
...           ...    ...     ...         ...           ...          ...   
9932         9726  23702     NaN   -0.552601           NaN          NaN   
9933         9729  29243     0.0   -0.634220     -0.417982    -0.642642   
9934         9734  28599     1.0   -0.307744     -0.417982    -0.135782   
9935         9734  27897     1.0   -0.428398           NaN          NaN   
9936         4202  25814     2.0   -0.795684           NaN          NaN   

      floor  rooms  layout   ceiling  ...  material  stories  \
0         3      6     NaN       Na

In [7]:
X_Train = XTrain.copy()
Y_Train = YTrain.copy()
X_Test = XTest.copy()

In [8]:
X = X_Train.copy()
y = Y_Train.copy()

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
# Template for replacing nan by computing other variables
#df["parking"] = df.groupby("district").transform(lambda x: x.fillna(x.median()))["parking"]

In [11]:
#df.column_name.loc[df.column_name > max_value]  = max_value

In [12]:
district_counts = {0:{},1:{},2:{},3:{},4:{},5:{},6:{},7:{},8:{},9:{},10:{},11:{},12:{}}
districts = XTrain["district"]
for d in districts:
    pass

In [13]:
pca_model = PCA(n_components=4)

In [14]:
pca_train = pca_model.fit_transform(X_train)

ValueError: could not convert string to float: 'Ломоносовский проспект'

In [ ]:
pca_train_Df = pd.DataFrame(data = pca_train
             , columns = ['principal 1', 'principal 2', 'principal 3', 'principal 4'])

In [ ]:
print('Explained variation per principal component: {}'.format(pca_model.explained_variance_ratio_))